In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import moviepy
import moviepy.editor as mp
import time
from IPython.display import clear_output
from tqdm import tqdm
from ultralytics import YOLO
import torch
import av
import numpy as np
import pandas as pd
from PIL import Image
import moviepy
import moviepy.editor as mp
from IPython.display import clear_output
import tempfile
import seaborn as sns
from transformers import DetrImageProcessor, DetrForObjectDetection
import cv2

# from IPython.display import Video

In [2]:
# Define a function to determine the state based on conditions
def determine_state(row):
    if row['Healthy']:
        return 'Healthy'
    elif row['Diarrhé'] and row['Pneumonie']:
        return 'Diarrhé_Pneumonie'
    elif row['Diarrhé']:
        return 'Diarrhé'
    elif row['Pneumonie']:
        return 'Pneumonie'
    else:
        return 'Unknown'

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"


# Function to sample uniformly
def uniform_sample(df, sample_size, groupby_cols):
    # Calculate the number of rows to sample per group
    num_groups = df.groupby(groupby_cols).ngroups
    sample_per_group = max(sample_size // num_groups, 1)
    
    # Sample uniformly within each group
    sampled_df = df.groupby(groupby_cols, group_keys=False).apply(lambda x: x.sample(min(len(x), sample_per_group)))
    
    # If we sampled less than the desired sample_size due to group sizes, sample more from the remaining dataframe
    # if len(sampled_df) < sample_size:
    #     remaining_sample_size = sample_size - len(sampled_df)
    #     remaining_df = df[~df.index.isin(sampled_df.index)]
    #     additional_samples = remaining_df.sample(min(len(remaining_df), remaining_sample_size))
    #     sampled_df = pd.concat([sampled_df, additional_samples])
    
    return sampled_df

# Function to sample uniformly with specified values
def uniform_sample_with_values(df, sample_size, groupby_cols, filter_values = None):
    copy = df.copy(deep=True)
    # Filter dataframe based on specified values for each column
    if filter_values is not None:
        for col, values in filter_values.items():
            copy = copy[copy[col].isin(values)]
    
    # Calculate the number of rows to sample per group
    num_groups = copy.groupby(groupby_cols).ngroups
    sample_per_group = max(sample_size // num_groups, 1)
    
    # Sample uniformly within each group
    sampled_df = copy.groupby(groupby_cols, group_keys=False).apply(lambda x: x.sample(min(len(x), sample_per_group)))
    
    # If we sampled less than the desired sample_size due to group sizes, sample more from the remaining dataframe
    if len(sampled_df) < sample_size:
        remaining_sample_size = sample_size - len(sampled_df)
        remaining_df = copy[~copy.index.isin(sampled_df.index)]
        additional_samples = remaining_df.sample(min(len(remaining_df), remaining_sample_size))
        sampled_df = pd.concat([sampled_df, additional_samples])
    
    return sampled_df, copy[~copy.index.isin(sampled_df.index)]

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    # start_idx = end_idx - converted_len
    # start_idx = clip_len
    start_idx = 0
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

def plot_results(pil_img, scores, boxes, labels, masks=None):
    plt.figure(figsize=(16,10))
    np_image = np.array(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if masks is None:
      masks = [None for _ in range(len(scores))]
        
    assert len(scores) == len(boxes) == len(labels) == len(masks)
    
    for s, (xmin, ymin, xmax, ymax), l, mask, c in zip(scores, boxes.tolist(), labels, masks, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{l}: {s:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))

        if mask is None:
          continue
        np_image = apply_mask(np_image, mask, c)

        padded_mask = np.zeros((mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
          verts = np.fliplr(verts) - 1
          p = Polygon(verts, facecolor="none", edgecolor=c)
          ax.add_patch(p)


    plt.imshow(np_image)
    plt.axis('off')
    plt.show()


def add_res(results, ax, color='green'):

    bboxes = results['boxes']
    labels = results['labels']
    scores = results['scores']

    colors = ['purple', 'yellow', 'red', 'green', 'orange', 'pink']
    
    for i, (b, ll, ss) in enumerate(zip(bboxes, labels, scores)):
        ax.add_patch(plt.Rectangle((b[0], b[1]), b[2] - b[0], b[3] - b[1], fill=False, color=colors[i], linewidth=3))
        cls_name = ll if isinstance(ll,str) else CLASSES[ll]
        text = f'{cls_name}: {ss:.2f}'
        print(text)
        ax.text(b[0], b[1], text, fontsize=15, bbox=dict(facecolor='white', alpha=0.8))


def plot_gallery(image_paths, n_cols=5):
    n_images = len(image_paths)
    n_rows = n_images // n_cols + int(n_images % n_cols > 0)
    
    plt.figure(figsize=(20, n_rows * 4))
    print(n_rows, n_cols)
    for i, image_path in enumerate(image_paths):
        img = Image.open(image_path)
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(img)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()   

def plot_bbox_gallery(image_df, n_cols=5):
    n_images = image_df.shape[0]
    n_rows = n_images // n_cols + int(n_images % n_cols > 0)
    
    plt.figure(figsize=(20, n_rows * 4))
    for i, row in image_df.iterrows():
        img = Image.open(row["path"])
        # plt.subplot(n_rows, n_cols, i + 1)
        ax = plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(img)
        # ax = plt.gca()
        score = row["conf"]
        box_x = row["box_x"]
        box_y = row["box_y"]
        box_width = row["box_width"]
        box_height = row["box_height"]
        box_x = box_x - (box_width / 2)
        box_y = box_y - (box_height / 2)
        rect = plt.Rectangle((box_x, box_y), box_width, box_height, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(box_x, box_y - 10, f' {score:.2f}', color='red', fontsize=12, backgroundcolor='white')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()   

In [3]:
def extract_videos(aptms, mp4_files, root_dir, frame_sample_rate = 4, num_images = 16, bbox_conf = .8, delta = 20, step = 0, model = None):
    
    size = len(aptms)
    records = []
    indices_records = []
    for idx in tqdm(range(size), desc="Processing items"):
        one_row = aptms.iloc[idx]
        target = one_row['State']
            
        cond = (
            (mp4_files["start_date"] == one_row.datetime.date())
            # (mp4_files["start_date"] == str(one_row.datetime.date()))
            & (mp4_files["start_time"] <= one_row.datetime)
            & (mp4_files["end_time"] > one_row.datetime)
            & (mp4_files["station"] == one_row.parc)
        )

        if mp4_files[cond].shape[0] == 0:
            continue

        dir_path = root_dir + "/" + target
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

        for index, file in mp4_files[cond].iterrows():
            zero = (one_row["datetime"] - file["start_time"]).total_seconds()
            if zero - delta > 0:
                break
    
        video = mp.VideoFileClip(file["path"])
        start_at = max(zero - delta, 0)
        end_at = max(zero - step, 0)
        
        # if end_at <= start_at or (end_at - start_at) <= 1:
        if end_at <= start_at or (end_at - start_at) < delta - step:
            continue

        # print(zero, start_at, end_at, file)
            
        clip = video.subclip(start_at, end_at)
        nfaces = 0
        
        if model:
            with tempfile.NamedTemporaryFile(suffix='.mp4') as tmpfile:
                clip.write_videofile(tmpfile.name, logger= None)
                container = av.open(tmpfile.name)
    
                video_stream = container.streams.video[0]
    
                # Get the duration in time base units
                duration_in_units = video_stream.duration
                
                # Get the time base
                time_base = video_stream.time_base
                
                # Calculate the duration in seconds
                duration_in_seconds = duration_in_units * time_base
    
                # Sample 16 frames
                fps = int(container.streams.video[0].average_rate)
                seg_len = int(duration_in_seconds) * fps
                sample_rate = min(int(seg_len // num_images), frame_sample_rate)
                indices = sample_frame_indices(clip_len=num_images, frame_sample_rate=sample_rate, seg_len=seg_len)
                video_frames = read_video_pyav(container, indices)
                video_images = [Image.fromarray(frame) for frame in video_frames]
                results = model.predict(video_images, save=False, imgsz=640, conf=bbox_conf, max_det=1, show=False)

                frames_idx = []
                for i, r in enumerate(results):
                    if r.boxes.shape[0] != 0:
                        nfaces += 1
                        frames_idx.append(indices[i])
                        
                    # else:
                        # nfaces = 0
                        
            # if nfaces < 1:
            # if nfaces < 5:
                # continue

        filename = f"clip_{idx}_{one_row['calfNumber']}_{one_row['parc']}_{one_row['datetime']}_{target}.mp4"
        # temp_path = f"{dir_path}/{filename}"
        temp_path = f"{dir_path}/{nfaces}_{filename}"
        clip.write_videofile(temp_path, logger= None)
        
        indices_records.extend([{"fps": fps, "seg_len": seg_len, "sample_rate": sample_rate, "idx": idx, "from": temp_path} for idx in frames_idx])
        
        records.append({
            "video": filename,
            "path": temp_path,
            "label": target,
            "duration": end_at - start_at,
            "calf": one_row["calfNumber"],
            "station": one_row["parc"],
            "before": one_row["datetime"],
            "from": file["path"],
            "nfaces": nfaces
        })
    
    clear_output()
    return pd.DataFrame(records), pd.DataFrame(indices_records)

def extract_medias(aptms, mp4_files, root_dir, frame_sample_rate = 4, num_images = 16, bbox_conf = (.8, .5), delta = 20, step = 0, video_model = None,  image_model = None):
    
    size = len(aptms)
    video_records = []
    image_records = []
    indices_records = []
    for idx in tqdm(range(size), desc="Processing items"):
        one_row = aptms.iloc[idx]
        target = one_row['State']
            
        cond = (
            (mp4_files["start_date"] == one_row.datetime.date())
            # (mp4_files["start_date"] == str(one_row.datetime.date()))
            & (mp4_files["start_time"] <= one_row.datetime)
            & (mp4_files["end_time"] > one_row.datetime)
            & (mp4_files["station"] == one_row.parc)
        )

        if mp4_files[cond].shape[0] == 0:
            continue

        video_dir_path = root_dir + "/videos/" + target
        if not os.path.exists(video_dir_path):
            os.makedirs(video_dir_path)

        image_dir_path = root_dir + "/images/" + target
        if not os.path.exists(image_dir_path):
            os.makedirs(image_dir_path)

        for index, file in mp4_files[cond].iterrows():
            zero = (one_row["datetime"] - file["start_time"]).total_seconds()
            if zero - delta > 0:
                break
    
        video = mp.VideoFileClip(file["path"])
        start_at = max(zero - delta, 0)
        end_at = max(zero - step, 0)
        
        # if end_at <= start_at or (end_at - start_at) <= 1:
        if end_at <= start_at or (end_at - start_at) < delta - step:
            continue

        # print(zero, start_at, end_at, file)
            
        clip = video.subclip(start_at, end_at)
        nfaces = 0
        
        with tempfile.NamedTemporaryFile(suffix='.mp4') as tmpfile:
            clip.write_videofile(tmpfile.name, logger= None)
            container = av.open(tmpfile.name)

            video_stream = container.streams.video[0]

            # Get the duration in time base units
            duration_in_units = video_stream.duration
            
            # Get the time base
            time_base = video_stream.time_base
            
            # Calculate the duration in seconds
            duration_in_seconds = duration_in_units * time_base

            # Sample 16 frames
            fps = int(container.streams.video[0].average_rate)
            seg_len = int(duration_in_seconds) * fps
            sample_rate = min(int(seg_len // num_images), frame_sample_rate)
            indices = sample_frame_indices(clip_len=num_images, frame_sample_rate=sample_rate, seg_len=seg_len)
            video_frames = read_video_pyav(container, indices)
            image_frames = [Image.fromarray(frame) for frame in video_frames]
        #     video_images = [Image.fromarray(frame) for frame in video_frames]
        #     results = video_model.predict(video_images, save=False, imgsz=640, conf=bbox_conf[1], max_det=1, show=False)

        #     frames_idx = []
        #     image_frames = []
        #     for i, r in enumerate(results):
        #         if r.boxes.shape[0] != 0:
        #             nfaces += 1
        #             frames_idx.append(indices[i])
        #             image_frames.append(video_images[i])
                    
        #         # else:
        #             # nfaces = 0
                    
        # if nfaces <= 1:
        # # if nfaces < 5:
        #     continue

        results = image_model.predict(image_frames, save=False, imgsz=640, conf=bbox_conf[0], max_det=1, show=False)
        result_records = []
        image_indices = []
        for frame_id, r in enumerate(results):
            if r.boxes.shape[0] == 0:
                continue
                # conf = 0
                # x1, y1, width, height = 0, 0, 0, 0
            else:
                conf = r.boxes.cpu().numpy().conf[0]
                x1, y1, width, height =  r.boxes.cpu().numpy().xywh[0]

            img = image_frames[frame_id]
            # filename = f"image_{frames_idx[frame_id]}_{idx}_{frame_id}_{one_row['calfNumber']}_{one_row['parc']}_{one_row['datetime']}_{target}.png"
            filename = f"image_{idx}_{frame_id}_{one_row['calfNumber']}_{one_row['parc']}_{one_row['datetime']}_{target}.png"
            img_save_path = os.path.join(image_dir_path, filename)
            img.save(img_save_path)
            # image_indices.append(frames_idx[frame_id])
            image_indices.append(indices[frame_id])

            result_records.append({
                "image": filename,
                "path": img_save_path,
                "label": target,
                "calf": one_row["calfNumber"],
                "station": one_row["parc"],
                "before": one_row["datetime"],
                "from": file["path"],
                "conf": conf, 
                "box_x": x1,
                "box_y": y1,
                "box_width": width,
                "box_height": height,
                # "passed": r.boxes.shape[0] == 0
            })

        # if len(result_records) == 0:
        if len(result_records) < 1:
            continue

        
        image_records.extend(result_records)
        filename = f"clip_{idx}_{one_row['calfNumber']}_{one_row['parc']}_{one_row['datetime']}_{target}.mp4"
        temp_path = f"{video_dir_path}/{filename}"
        # temp_path = f"{dir_path}/{nfaces}_{filename}"
        clip.write_videofile(temp_path, logger= None)
        
        indices_records.extend([{"fps": fps, "seg_len": seg_len, "sample_rate": sample_rate, "idx": idx, "from": temp_path} for idx in image_indices])
        
        video_records.append({
            "video": filename,
            "path": temp_path,
            "label": target,
            "duration": end_at - start_at,
            "calf": one_row["calfNumber"],
            "station": one_row["parc"],
            "before": one_row["datetime"],
            "from": file["path"],
            "nfaces": nfaces
        })
    
    clear_output()
    return pd.DataFrame(video_records), pd.DataFrame(image_records), pd.DataFrame(indices_records)


In [4]:
ROOT_DIR = "/data/konrad/workspace"
yolo_face = YOLO(ROOT_DIR + "/models/best-face3.pt")
yolo_world = YOLO("yolov8s-world.pt")  
yolo_world.set_classes(["calf face"])

# yolo, _ = torch.hub.load('ashkamath/mdetr:main', 'mdetr_efficientnetB5', pretrained=True, return_postprocessor=True, trust_repo=True, force_reload=True)
# yolo = mdter.cuda()
# yolo.eval()

# model_name = 'ashkamath/mdetr-resnet50'
# mdter_postprocessor = DetrImageProcessor.from_pretrained(model_name)
# mdter = DetrForObjectDetection.from_pretrained(model_name)

In [5]:
# Read the CSV file into a DataFrame
states = pd.read_csv(ROOT_DIR +'/csv_files/illness.csv')
aptm = pd.read_csv(ROOT_DIR + '/csv_files/visits_point.csv', delimiter=';')

DATA_DIR = "/data/public/heureux_2022-02-16_2022-04-16"
dir_path = DATA_DIR
directories = [d for d in os.listdir(dir_path) if d.startswith("2022") and os.path.isdir(os.path.join(dir_path, d))]

day_to_dir = {}
for d in directories:
    if d.startswith("2022-"):
        p = d.split("-")
        start_date = "-".join(p[:3])
        end_date = "-".join(p[3:])
        date_range = pd.date_range(start=start_date, end=end_date)
        day_to_dir.update({md:d for md in date_range.strftime('%Y-%m-%d').tolist()})

day_to_dir['2022-02-02'] = "2022_02_02"

mp4_files = []
for d in directories:
    try:
        path = dir_path + "/" + d
        # print(path)
        for f in os.listdir(path):
            if f.endswith(".mp4") and os.path.isfile(os.path.join(path, f)):
                filename = f[:-4].split("_")
                start_date = filename[3][:4] + "-" + filename[3][4:6] + "-" + filename[3][6:8] + "T" + filename[3][8:10] + ":" + filename[3][10:12] + ":" + filename[3][12:]
                end_date = filename[4][:4] + "-" + filename[4][4:6] + "-" + filename[4][6:8] + "T" + filename[4][8:10] + ":" + filename[4][10:12] + ":" + filename[4][12:]
                station = filename[1][2:]
                # print(filename, start_date, end_date, station)
                # print(os.path.join(path, f))
                mp4_files.append({"station": station, "start_at": start_date, "end_at": end_date, "path": os.path.join(path, f)})
                # break
            
    except Exception as e:
        continue

mp4_files = pd.DataFrame(mp4_files)
mp4_files["start_time"] = pd.to_datetime(mp4_files["start_at"])
mp4_files["end_time"] = pd.to_datetime(mp4_files["end_at"])
mp4_files["start_date"] = mp4_files["start_time"].dt.date
mp4_files["start_hour"] = mp4_files["start_time"].dt.hour
mp4_files["start_mn"] = mp4_files["start_time"].dt.minute
mp4_files = mp4_files[mp4_files["station"].isin(["1","2","9","10"])]

In [6]:
states['State'] = states.apply(determine_state, axis=1)
states = states[~states["State"].isna()]

columns = ["calfNumber", "station", "Duration", "localDate", "date", "feederLong"]
aptm["datetime"] = pd.to_datetime(aptm["localDate"])
aptm["day"] = aptm["datetime"].dt.date
aptm["hour"] = aptm["datetime"].dt.time

# Only data for eval date
stated_aptm = aptm[aptm["day"].apply(str).isin(states.date.unique())][columns + ["day", "hour", "datetime"]]
stated_aptm["parc"] = None
stated_aptm.loc[(stated_aptm["station"]==1) & (stated_aptm["feederLong"]=="DAL 2 (2496)"), 'parc']="9" #louve 3
stated_aptm.loc[(stated_aptm["station"]==2) & (stated_aptm["feederLong"]=="DAL 2 (2496)"), 'parc']="10" #louve 4
stated_aptm.loc[(stated_aptm["station"]==2) & (stated_aptm["feederLong"]=="DAL 1 (2494)"), 'parc']="1"  #louve 2 
stated_aptm.loc[(stated_aptm["station"]==1) & (stated_aptm["feederLong"]=="DAL 1 (2494)"), 'parc']="2" #louve 1
stated_aptm = stated_aptm[stated_aptm["Duration"] > 0]
stated_aptm = stated_aptm[~stated_aptm["parc"].isna()]
# Only data for available recorded video dates
stated_aptm = stated_aptm[stated_aptm["day"].apply(str).isin(day_to_dir.keys())]
# Only apptm for available recorded video hour
stated_aptm = stated_aptm[(stated_aptm["datetime"].dt.hour >= 6) & (stated_aptm["datetime"].dt.hour <= 22)]
stated_aptm["cwd"] = stated_aptm["calfNumber"].astype(str) + " " + stated_aptm["day"].astype(str)
states["cwd"] = states["No ATQ"].astype(str) + " " + states["date"].astype(str)
aptm_with_state = pd.merge(stated_aptm[["calfNumber", "parc", "day", "Duration", "cwd", "datetime"]], states[["State", "cwd"]], on="cwd", how='inner')

drop_idx=[]
visited_day = {}
minutes = 3
for idx in aptm_with_state.sort_values(by="datetime").index.values:
    key = "{day}_{parc}".format(day=aptm_with_state.iloc[idx]["day"], parc=aptm_with_state.iloc[idx]["parc"])

    if not key in visited_day.keys():
        visited_day[key] = aptm_with_state.iloc[idx]["datetime"] +  pd.Timedelta(seconds=aptm_with_state.iloc[idx]["Duration"])
        continue
    
    diff = (aptm_with_state.iloc[idx]["datetime"] - visited_day[key]).total_seconds()
    
    if diff < (minutes * 60):
        drop_idx.append(idx)
    else:
        visited_day[key] = aptm_with_state.iloc[idx]["datetime"]

spaced_aptm_with_state = aptm_with_state[~aptm_with_state.index.isin(drop_idx)]
spaced_aptm_with_state.reset_index(drop=True, inplace=True)

# aptm_with_state.to_csv('aptm_with_state.csv', index=False)
# spaced_aptm_with_state.to_csv('spaced_aptm_with_state.csv', index=False)
# mp4_files.to_csv('mp4_files.csv', index=False)

In [7]:
aptm_sampled = spaced_aptm_with_state[spaced_aptm_with_state["State"].isin(['Diarrhé', 'Pneumonie', 'Healthy', 'Diarrhé, Pneumonie'])]

In [ ]:
# Start time
start_time = time.time()

frame = 20
step = 0
frame_sample_rate = 4
num_images = 40
ibbox_conf = .80
vbbox_conf = .50
delta = frame + step
# root_dir = ROOT_DIR + f"/datasets/mixed_{frame}s_b{step}s"
root_dir = f"/data/data_calves/konrad/mixed_{frame}s_b{step}s"
# root_dir = ROOT_DIR + "/datasets/videos/train"

records = extract_medias(aptm_sampled, mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, video_model = yolo_world, image_model = yolo_face, bbox_conf = (ibbox_conf, vbbox_conf))
# records = extract_medias(aptm_sampled.iloc[:100], mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, video_model = yolo_world, image_model = yolo_face, bbox_conf = (ibbox_conf, vbbox_conf))
# records = extract_videos_mdetr(aptm_sampled.iloc[:100], mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, model = yolo, bbox_conf = bbox_conf)
# records = extract_videos(aptm_sampled.iloc[:100], mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, model = yolo, bbox_conf = bbox_conf)
# records = extract_videos(aptm_sampled, mp4_files, root_dir, delta = delta, step = step, frame_sample_rate = frame_sample_rate, num_images = num_images, model = yolo, bbox_conf = bbox_conf)
# records = extract_videos(aptm_sampled.iloc[:10], mp4_files, root_dir, delta = delta, step = step)
# records = extract_videos(aptm_sampled, mp4_files, root_dir, delta = delta, step = step)

# End time
end_time = time.time()

# Calculate elapsed time
elapsed_time = end_time - start_time

formatted_time = format_time(elapsed_time)
records[0].to_csv(root_dir + '/videos_metadata.csv', index=False)
records[1].to_csv(root_dir + '/images_metadata.csv', index=False)
records[2].to_csv(root_dir + '/indices_metadata.csv', index=False)
# records.to_csv('train_video_extracted_metadata.csv', index=False)

# Display the elapsed time
print(f"Time of execution: {formatted_time} from {elapsed_time}")

Processing items:   0%|                                                                                                                      | 0/3854 [00:00<?, ?it/s]


0: 384x640 (no detections), 1.5ms
1: 384x640 (no detections), 1.5ms
2: 384x640 (no detections), 1.5ms
3: 384x640 (no detections), 1.5ms
4: 384x640 (no detections), 1.5ms
5: 384x640 (no detections), 1.5ms
6: 384x640 (no detections), 1.5ms
7: 384x640 (no detections), 1.5ms
8: 384x640 (no detections), 1.5ms
9: 384x640 (no detections), 1.5ms
10: 384x640 (no detections), 1.5ms
11: 384x640 (no detections), 1.5ms
12: 384x640 (no detections), 1.5ms
13: 384x640 (no detections), 1.5ms
14: 384x640 (no detections), 1.5ms
15: 384x640 (no detections), 1.5ms
16: 384x640 (no detections), 1.5ms
17: 384x640 (no detections), 1.5ms
18: 384x640 (no detections), 1.5ms
19: 384x640 (no detections), 1.5ms
20: 384x640 (no detections), 1.5ms
21: 384x640 (no detections), 1.5ms
22: 384x640 (no detections), 1.5ms
23: 384x640 (no detections), 1.5ms
24: 384x640 (no detections), 1.5ms
25: 384x640 (no detections), 1.5ms
26: 384x640 (no detections), 1.5ms
27: 384x640 (no detections), 1.5ms
28: 384x640 (no detections), 

Processing items:   0%|                                                                                                            | 1/3854 [00:08<8:47:36,  8.22s/it]


0: 384x640 (no detections), 0.3ms
1: 384x640 1 Face, 0.3ms
2: 384x640 1 Face, 0.3ms
3: 384x640 1 Face, 0.3ms
4: 384x640 1 Face, 0.3ms
5: 384x640 1 Face, 0.3ms
6: 384x640 (no detections), 0.3ms
7: 384x640 (no detections), 0.3ms
8: 384x640 (no detections), 0.3ms
9: 384x640 1 Face, 0.3ms
10: 384x640 1 Face, 0.3ms
11: 384x640 1 Face, 0.3ms
12: 384x640 1 Face, 0.3ms
13: 384x640 1 Face, 0.3ms
14: 384x640 1 Face, 0.3ms
15: 384x640 (no detections), 0.3ms
16: 384x640 (no detections), 0.3ms
17: 384x640 (no detections), 0.3ms
18: 384x640 (no detections), 0.3ms
19: 384x640 (no detections), 0.3ms
20: 384x640 (no detections), 0.3ms
21: 384x640 (no detections), 0.3ms
22: 384x640 (no detections), 0.3ms
23: 384x640 (no detections), 0.3ms
24: 384x640 1 Face, 0.3ms
25: 384x640 1 Face, 0.3ms
26: 384x640 1 Face, 0.3ms
27: 384x640 1 Face, 0.3ms
28: 384x640 1 Face, 0.3ms
29: 384x640 1 Face, 0.3ms
30: 384x640 (no detections), 0.3ms
31: 384x640 (no detections), 0.3ms
32: 384x640 (no detections), 0.3ms
33: 384

Processing items:   0%|                                                                                                           | 2/3854 [00:32<18:51:09, 17.62s/it]

...

Processing items:  27%|███████████████████████████▎                                                                        | 1051/3854 [4:31:09<151:35:44, 194.70s/it]


0: 384x640 (no detections), 0.3ms
1: 384x640 (no detections), 0.3ms
2: 384x640 (no detections), 0.3ms
3: 384x640 (no detections), 0.3ms
4: 384x640 (no detections), 0.3ms
5: 384x640 (no detections), 0.3ms
6: 384x640 (no detections), 0.3ms
7: 384x640 (no detections), 0.3ms
8: 384x640 (no detections), 0.3ms
9: 384x640 (no detections), 0.3ms
10: 384x640 (no detections), 0.3ms
11: 384x640 (no detections), 0.3ms
12: 384x640 (no detections), 0.3ms
13: 384x640 (no detections), 0.3ms
14: 384x640 (no detections), 0.3ms
15: 384x640 (no detections), 0.3ms
16: 384x640 (no detections), 0.3ms
17: 384x640 (no detections), 0.3ms
18: 384x640 (no detections), 0.3ms
19: 384x640 (no detections), 0.3ms
20: 384x640 (no detections), 0.3ms
21: 384x640 (no detections), 0.3ms
22: 384x640 (no detections), 0.3ms
23: 384x640 (no detections), 0.3ms
24: 384x640 (no detections), 0.3ms
25: 384x640 (no detections), 0.3ms
26: 384x640 (no detections), 0.3ms
27: 384x640 (no detections), 0.3ms
28: 384x640 (no detections), 

Processing items:  27%|███████████████████████████▎                                                                        | 1052/3854 [4:34:43<155:58:13, 200.39s/it]


0: 384x640 (no detections), 0.3ms
1: 384x640 (no detections), 0.3ms
2: 384x640 (no detections), 0.3ms
3: 384x640 (no detections), 0.3ms
4: 384x640 (no detections), 0.3ms
5: 384x640 (no detections), 0.3ms
6: 384x640 (no detections), 0.3ms
7: 384x640 (no detections), 0.3ms
8: 384x640 (no detections), 0.3ms
9: 384x640 (no detections), 0.3ms
10: 384x640 (no detections), 0.3ms
11: 384x640 (no detections), 0.3ms
12: 384x640 (no detections), 0.3ms
13: 384x640 (no detections), 0.3ms
14: 384x640 (no detections), 0.3ms
15: 384x640 (no detections), 0.3ms
16: 384x640 (no detections), 0.3ms
17: 384x640 (no detections), 0.3ms
18: 384x640 (no detections), 0.3ms
19: 384x640 (no detections), 0.3ms
20: 384x640 (no detections), 0.3ms
21: 384x640 (no detections), 0.3ms
22: 384x640 (no detections), 0.3ms
23: 384x640 (no detections), 0.3ms
24: 384x640 (no detections), 0.3ms
25: 384x640 (no detections), 0.3ms
26: 384x640 (no detections), 0.3ms
27: 384x640 (no detections), 0.3ms
28: 384x640 (no detections), 

Processing items:  27%|███████████████████████████▉                                                                          | 1057/3854 [4:34:50<50:17:35, 64.73s/it]